
*Note:* You can run this from your computer (Jupyter or terminal), or use one of the
hosted options:

[![binder-logo](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ploomber/binder-env/main?urlpath=git-pull%3Frepo%3Dhttps%253A%252F%252Fgithub.com%252Fploomber%252Fprojects%26urlpath%3Dlab%252Ftree%252Fprojects%252Flogging%252FREADME.ipynb%26branch%3Dmaster)

[![deepnote-logo](https://deepnote.com/buttons/launch-in-deepnote-small.svg)](https://deepnote.com/launch?template=deepnote&url=https://github.com/ploomber/projects/blob/master/logging/README.ipynb)


In [1]:
from ploomberutils import display_file

# Logging

This guide will show you how to log pipeline execution.

The *Summary* section provides a quick reference for configuring logging. If you want the details, continue reading.

## Summary

### Function tasks

If you're using functions as tasks, configure logging like this:

```python
import logging


def some_task(product):
    # uncomment the next line if using Python >= 3.8 on macOS
    # logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(__name__)

    # to log a message, call logger.info
    logger.info('Some message')
```

### Scripts or notebooks

If using scripts/notebooks tasks, add this a the top of **each** one:

```python
import sys
import logging

logging.basicConfig(level=logging.INFO, stream=sys.stdout)
logger = logging.getLogger(__name__)

# to log a message, call logger.info
logger.info('Some message')
```

**and** add the following to **each** task definition:

```yaml
tasks:
  - source: scripts/script.py
    product: products/output.ipynb
    # add this
    papermill_params:
      log_output: True
```

Then, use the `--log` option when building the pipeline:

```sh
ploomber build --log info
```

## Sample pipeline

The pipeline we'll be using for this guide contains two tasks (a script and a function):

In [2]:
display_file('basic/pipeline.yaml')


```yaml
tasks:
  - source: script.py
    product: output/nb-log.ipynb
    papermill_params:
      log_output: True

  - source: tasks.function
    product: output/fn-log.txt
```


Note that the script task contains:

```yaml
papermill_params:
    log_output: True
```

This extra configuration is required on each script/notebook task in your pipeline to enable logging. The code on each task isn't important, they contain a for loop and log a message on each iteration. Let's see it in action:

In [3]:
%%sh
cd basic
ploomber build --log info --force

name      Ran?      Elapsed (s)    Percentage
--------  ------  -------------  ------------
script    True          4.95027       62.2437
function  True          3.00278       37.7563


INFO:blib2to3.pgen2.driver:Generating grammar tables from /Users/Edu/miniconda3/envs/projects/lib/python3.7/site-packages/blib2to3/Grammar.txt
INFO:blib2to3.pgen2.driver:Writing grammar tables to /Users/Edu/Library/Caches/black/21.8b0/Grammar3.7.10.final.0.pickle
INFO:blib2to3.pgen2.driver:Writing failed: [Errno 2] No such file or directory: '/Users/Edu/Library/Caches/black/21.8b0/tmpeoom9bxf'
INFO:blib2to3.pgen2.driver:Generating grammar tables from /Users/Edu/miniconda3/envs/projects/lib/python3.7/site-packages/blib2to3/PatternGrammar.txt
INFO:blib2to3.pgen2.driver:Writing grammar tables to /Users/Edu/Library/Caches/black/21.8b0/PatternGrammar3.7.10.final.0.pickle
INFO:blib2to3.pgen2.driver:Writing failed: [Errno 2] No such file or directory: '/Users/Edu/Library/Caches/black/21.8b0/tmp1ifxyuzd'
INFO:ploomber.dag.dag:Building DAG DAG("No name")
Building task 'script':   0%|          | 0/2 [00:00<?, ?it/s]INFO:ploomber.tasks.abc.NotebookRunner:Starting execution: NotebookRunner: script

We can see that the logging statements appear in the console. If you want to take a look at the code, [click here](https://github.com/ploomber/projects/tree/master/guides/logging/basic).

## Why not print?

Note that the snippets above use the `logging` module instead of `print`. Although `print` is a quick and easy way to display messages in the console, the `logging` module is more flexible. Hence, it is the recommended option.

## Logging to a file (On Linux and macOS)

It's common to send all your log records to a file. You can easily do so on Linux and macOS with the following command:

```sh
ploomber build --log info > my.log 2>&1
```

## Logging to a file from Python (Linux, macOS, and Windows)

Alternatively, you can configure logging from Python, which gives you more flexibility:

In [4]:
# you may store the contents of this cell in a .py file and then call it from the command line
# e.g., python run_pipeline.py
import logging
from pathlib import Path

from ploomber.spec import DAGSpec

logging.basicConfig(filename='my.log', format='%(levelname)s:%(message)s', level=logging.INFO)

dag = DAGSpec('basic/pipeline.yaml').to_dag()
dag.build(force=True)

  0%|          | 0/2 [00:00<?, ?it/s]

Executing:   0%|          | 0/6 [00:00<?, ?cell/s]

name,Ran?,Elapsed (s),Percentage
script,True,4.89868,61.9864
function,True,3.00415,38.0136


Let's look at the file contents:

In [5]:
print(Path('my.log').read_text())

INFO:Generating grammar tables from /Users/Edu/miniconda3/envs/projects/lib/python3.7/site-packages/blib2to3/Grammar.txt
INFO:Writing grammar tables to /Users/Edu/Library/Caches/black/21.8b0/Grammar3.7.10.final.0.pickle
INFO:Writing failed: [Errno 2] No such file or directory: '/Users/Edu/Library/Caches/black/21.8b0/tmp6sx5clyw'
INFO:Generating grammar tables from /Users/Edu/miniconda3/envs/projects/lib/python3.7/site-packages/blib2to3/PatternGrammar.txt
INFO:Writing grammar tables to /Users/Edu/Library/Caches/black/21.8b0/PatternGrammar3.7.10.final.0.pickle
INFO:Writing failed: [Errno 2] No such file or directory: '/Users/Edu/Library/Caches/black/21.8b0/tmpwz6y5btf'
INFO:Building DAG DAG("No name")
INFO:Starting execution: NotebookRunner: script -> File('basic/output/nb-log.ipynb')
INFO:Input Notebook:  /var/folders/3h/_lvh_w_x5g30rrjzb_xnn2j80000gq/T/tmpjndfw52j.ipynb
INFO:Output Notebook: /Users/Edu/dev/projects-ploomber/logging/basic/output/nb-log.ipynb
INFO:Executing notebook with

## Controlling logging level

The Python's [logging](https://docs.python.org/3/library/logging.html) module allows you to filter messages depending on their priority. For example, when running your pipeline, you may only want to display *regular* messages, but you may allow *regular* and *debugging* messages for more granularity when debugging. Since Ploomber runs tasks differently depending on their type (i.e., functions vs. scripts/notebooks), controlling the logging level requires a bit of extra work. Let's use the same pipeline in the `parametrized` directory:

In [6]:
%%sh
cd parametrized
ploomber build --log info --env--logging_level info --force

name      Ran?      Elapsed (s)    Percentage
--------  ------  -------------  ------------
script    True          4.92702       62.1303
function  True          3.00312       37.8697


INFO:blib2to3.pgen2.driver:Generating grammar tables from /Users/Edu/miniconda3/envs/projects/lib/python3.7/site-packages/blib2to3/Grammar.txt
INFO:blib2to3.pgen2.driver:Writing grammar tables to /Users/Edu/Library/Caches/black/21.8b0/Grammar3.7.10.final.0.pickle
INFO:blib2to3.pgen2.driver:Writing failed: [Errno 2] No such file or directory: '/Users/Edu/Library/Caches/black/21.8b0/tmp8ewc2h7q'
INFO:blib2to3.pgen2.driver:Generating grammar tables from /Users/Edu/miniconda3/envs/projects/lib/python3.7/site-packages/blib2to3/PatternGrammar.txt
INFO:blib2to3.pgen2.driver:Writing grammar tables to /Users/Edu/Library/Caches/black/21.8b0/PatternGrammar3.7.10.final.0.pickle
INFO:blib2to3.pgen2.driver:Writing failed: [Errno 2] No such file or directory: '/Users/Edu/Library/Caches/black/21.8b0/tmpdgyq7d8x'
INFO:ploomber.dag.dag:Building DAG DAG("No name")
Building task 'script':   0%|          | 0/2 [00:00<?, ?it/s]INFO:ploomber.tasks.abc.NotebookRunner:Starting execution: NotebookRunner: script

Let's now run the pipeline but switch the logging level to debug, this will print the records we saw above, plus the ones with `debug` level:

In [7]:
%%sh
cd parametrized
ploomber build --log debug --env--logging_level debug --force

name      Ran?      Elapsed (s)    Percentage
--------  ------  -------------  ------------
script    True           4.5976        60.478
function  True           3.0045        39.522


DEBUG:ploomber.spec.dagspec:DAGSpec enviroment:
EnvDict({'cwd': '/Users/Edu/d.../parametrized', 'here': '/Users/Edu/d.../parametrized', 'logging_level': 'debug', 'root': '/Users/Edu/d.../parametrized', ...})
DEBUG:ploomber.spec.dagspec:Expanded DAGSpec:
{   'tasks': [   {   'papermill_params': {'log_output': True},
                     'params': {'logging_level': '{{logging_level}}'},
                     'product': 'output/nb-log.ipynb',
                     'source': 'script.py'},
                 {   'params': {'logging_level': '{{logging_level}}'},
                     'product': 'output/fn-log.txt',
                     'source': 'tasks.function'}]}
DEBUG:ploomber.tasks.abc.NotebookRunner:Setting "script" status to TaskStatus.WaitingRender
DEBUG:ploomber.tasks.abc.PythonCallable:Setting "function" status to TaskStatus.WaitingRender
DEBUG:ploomber.spec.dagspec:Extracted upstream dependencies for task script: None
DEBUG:ploomber.spec.dagspec:Extracted upstream dependencies for task 

To get the code for the previous example, [click here](https://github.com/ploomber/projects/tree/master/guides/logging/parametrized).

## Implementation details

To keep the tutorial short, we overlooked some technical details. However, if you want to customize logging, they are important to know.

### Function tasks and sub-processes

By default, Ploomber runs function tasks in a child process. However, beginning on version 3.8, [Python 3.8 switched to use spawn instead of fork on macOS](https://docs.python.org/3/library/multiprocessing.html#contexts-and-start-methods), this implies that child processes *do not* inherit the logging configuration of their parents. That's why you must configure a logger inside the function's body:

```python
import logging


def some_task(product):
    # the following line is required on Python>=3.8 if using macOS
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(__name__)

    # to log a message, call logger.info
    logger.info('Some message')
```

### Scripts and notebooks

Unlike function tasks, which can run in the same process that runs Ploomber, or in a child process, scripts and notebooks execute independently. Hence, any logging configuration made in the main process is lost, and We have to configure a separate logger at the top of the script/notebook.

### Parallel execution


Logging is currently unavailable when using the `Parallel` executor.
